In [20]:
import os 
import cv2
import time
import uuid

In [21]:
IMAGES_PATH = os.path.join('data','images')
number_images = 30

In [22]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [5]:
!pip install labelme


     ---------------------------------------- 1.5/1.5 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.7/7.7 MB 12.3 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 11.5 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB ? eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB 5.3 MB/s eta 0:00:00
  Created wheel for labelme: filename=labelme-5.3.1-py3-none-any.whl size=1472298 sha256=20226734d54490277cdb784f5ff50fac161ab91446401ce01ea3ef71c48c4945
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\06\1a\bf\efe61d1090b71eab97143593c8116868cefdab69128dd8ef1f
Successfully built labelme


In [23]:
!labelme

[INFO   ] __init__:get_config:70 - Loading config file from: C:\Users\user\.labelmerc
   QT_AUTO_SCREEN_SCALE_FACTOR to enable platform plugin controlled per-screen factors.
   QT_SCREEN_SCALE_FACTORS to set per-screen DPI.
   QT_SCALE_FACTOR to set the application global scale factor.


In [15]:
length_train = 90
length_test = length_val = 15
def split_images(folder,length):
    i = 0
    for file in os.listdir(os.path.join('data', 'images')):
        filename = file.split('.')[0]+'.jpg'
        existing_filepath = os.path.join('data','images', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'images',filename)
            os.replace(existing_filepath, new_filepath)
        if i== length:
            break
        i+=1

In [24]:
split_images(folder='train',length=30)

In [17]:
split_images(folder='test',length=length_test)

In [18]:
split_images(folder='val',length=length_val)

In [25]:
for folder in ['train']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath)  

In [28]:
from tensorflow.keras.models import load_model
facetracker = load_model('facetracker.h5')

In [31]:
import tensorflow as tf
import numpy as np
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 168ms/step


1/1 [==============================] - 0s 162ms/step
